<a href="https://colab.research.google.com/github/Fatima-Zahra7/ESILV-Cryptographie/blob/main/crypto_De_Sousa_Bekraoui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PARTIE 1**

1.   Class PasswordHash

Hashing Passwords: The hash_password method takes a plaintext password and a salt as input, hashes the password using the Argon2 algorithm, and returns the hashed password. It concatenates the password and salt before hashing to increase security.

Verifying Passwords: The verify_password method compares a hashed password stored in the database with a provided plaintext password. It takes the hashed password, salt, and provided password as input, verifies if the provided password matches the hashed one using Argon2, and returns a boolean indicating whether the verification succeeded or failed.

Error Handling: The verify_password method catches exceptions raised during the verification process, such as argon2.exceptions.VerifyMismatchError, and returns False if the provided password does not match the hashed one.

2.   Class User

id: An identifier for the user.
username: The username associated with the user.
password: The hashed password of the user.
salt: The salt used in conjunction with the password hashing.
key: A cryptographic key associated with the user, potentially used for encryption or other cryptographic operations.

3.   Class UserManager

User Registration: The register_user method allows registering a new user with a unique username and a strong password. It generates a user ID, hashes the password with a salt for security, creates a keyset for encryption purposes, and saves the user's information to a database and a file.

Password Strength Checking: The is_password_strong method checks if a given password meets certain strength criteria, including length, presence of digits, uppercase and lowercase letters, and special characters.

User Login: The login_user method verifies a user's credentials during login. It retrieves the user's stored password hash and salt from the database, hashes the provided password, and compares it with the stored hash to authenticate the user.

User Information Retrieval: The get_user_key_salt method retrieves the key, salt, and password hash associated with a given username from the database.

User Logout: The logout_user method resets the current user's session, effectively logging them out of the system.

4.   Class SaltGenerator

Generate Salt: The generate_salt method generates a random salt of a specified length. It utilizes Python's random.choice function to select characters randomly from a pool of ASCII letters and digits. The length of the salt is set to 16 characters by default.
The salt generated by this method can be used to enhance the security of password hashing by adding randomness to the hashing process, thereby mitigating the risk of precomputed hash attacks.

5.   Class UserDBManager

Initialization: Upon initialization, it loads user data from a specified file into memory.
Load Users: The load_users method reads user data from a CSV file and populates the users dictionary attribute with User objects.
Save User to File: The save_user_to_file method appends user data to the CSV file. If the file is empty, it writes the header first.

6.   Class KeyGenerator

Initialization: Upon initialization, the KeyGenerator class attempts to create a new keyset handle using the create_keyset method. If successful, it stores the keyset handle for later use.

Keyset Creation: The create_keyset method registers the Deterministic Authenticated Encryption with Associated Data (DÆAD) primitive and creates a new keyset handle using the AES256 SIV key template provided by Tink. This keyset handle can then be used for encryption and decryption operations.

Error Handling: The class includes error handling to catch any exceptions that may occur during key generation or initialization. If an error occurs, it prints the error message and raises an exception to notify the caller.



In [ ]:
import argon2

class PasswordHasher:

    def hash_password(self, password, salt):
        # Hacher le mot de passe avec Argon2
        hasher = argon2.PasswordHasher()

        # Salt le mot de passe
        hashed_password = hasher.hash(password + salt)
        return hashed_password

#****************************************

    def verify_password(self, hashed_password, salt, provided_password):
        # Vérifier si le mot de passe fourni correspond au mot de passe haché
        hasher = argon2.PasswordHasher()
        try:
            # Tenter de vérifier le mot de passe
            hasher.verify(hashed_password, provided_password + salt)
            return True
        except argon2.exceptions.VerifyMismatchError:
            return False

In [ ]:
class SaltGenerator :
    @staticmethod
    def generate_salt():
        # Génère un sel aléatoire
        length = 16  # Longueur du sel
        chars = string.ascii_letters + string.digits
        return ''.join(random.choice(chars) for _ in range(length))

In [ ]:
!pip install tink

In [ ]:
import tink
from tink import daead

class KeyGenerator:
    def __init__(self):
        try:
            tink_keyset_handle = self.create_keyset()
            self.tink_keyset_handle = tink_keyset_handle
        except Exception as e:
            print(e)
            raise Exception("Error while initializing KeyGenerator")

    def create_keyset(self):
        try:
            daead.register()
            keyset_handle = tink.new_keyset_handle(daead.deterministic_aead_key_templates.AES256_SIV)
            return keyset_handle
        except Exception as e:
            print(e)
            raise Exception("Error while creating keyset")


In [ ]:
class User:
    def __init__(self, id, username, password, salt, key):
        self.id = id
        self.username = username
        self.password = password
        self.salt = salt
        self.key = key

In [ ]:
import csv
import os

class UserDBManager:

    def __init__(self, filename):
        self.users = {}
        self.load_users(filename)

    def load_users(self, filename):
        if os.path.isfile(filename) and os.path.getsize(filename) > 0:
            with open(filename, 'r') as file:
                reader = csv.reader(file)
                header = next(reader)

                if header == ['ID', 'Username', 'Password', 'Salt', 'Key']:
                    for row in reader:
                        if row:  # Vérifier si la ligne est valide

                            user_id = int(row[0])
                            self.users[user_id] = User(user_id, row[1], row[2], row[3], row[4])


    def save_user_to_file(self, user, filename):
        if not user:
            return

        with open(filename, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)

            if csvfile.tell() == 0:
                writer.writerow(['ID', 'Username', 'Password', 'Salt', 'Key'])

            writer.writerow([user.id, user.username, user.password, user.salt, user.key])
        print("User " + user.username + " was successfully registered.")


In [ ]:
import random
import string


class UserManager:
    def __init__(self, db_manager):
        self.db_manager = db_manager

    def register_user(self, username, password, filename):

      # Vérifier si des utilisateurs sont déjà enregistrés
      if not self.db_manager or not self.db_manager.users:
          user_id = 1  # Si aucun utilisateur n'est enregistré, l'ID du premier utilisateur sera 1
      else:
          # Trouver l'ID le plus élevé parmi les utilisateurs existants et l'incrémenter de 1
          user_id = max(user.id for user in self.db_manager.users.values()) + 1

          # Vérifier si le nom d'utilisateur est déjà pris
          for user in self.db_manager.users.values():
              if user.username == username:
                  print("This username is taken.")
                  return

      # Assurer la robustesse du mot de passe
      while not self.is_password_strong(password):
          password = input("Enter a more secure password: ")

      # Générer une clé pour l'utilisateur
      keyset_handle = KeyGenerator().create_keyset()

      # Hasher le mot de passe
      salt = SaltGenerator().generate_salt()
      hashed_password = PasswordHasher().hash_password(password, salt)

      # Créer un nouvel utilisateur
      new_user = User(user_id, username, hashed_password, salt, keyset_handle)

      # Ajouter le nouvel utilisateur à la base de données
      self.db_manager.users[user_id] = new_user

      # Enregistrer le nouvel utilisateur dans un fichier
      self.db_manager.save_user_to_file(new_user, filename)


    def is_password_strong(self, password):
        if len(password) < 14:
            return False

        has_digit = any(char.isdigit() for char in password)
        has_upper = any(char.isupper() for char in password)
        has_lower = any(char.islower() for char in password)
        has_special = any(char in "!@#$%^&*()-_=+[]{}|;:'\",.<>/?`~" for char in password)

        return has_digit and has_upper and has_lower and has_special

#*******************************************************************************

    def login_user(self, username, password):
        password_hasher = PasswordHasher()
        user_info = self.get_user_key_salt(username)

        if user_info:
            key, salt, db_password = user_info

            hashed_password = PasswordHasher().hash_password(password, salt)

            if password_hasher.verify_password(hashed_password, salt, password):
                print("Connexion réussie!")
                return True

            else:
                print("Password incorrect!")
                return False
        else:
            print("User not found.")
            return False


    def get_user_key_salt(self, username):
      # Vérifier si des utilisateurs sont enregistrés

      if self.db_manager.users:
        for user_id, user in self.db_manager.users.items():
          if user.username == username:
            return user.key, user.salt, user.password

      print("There is no users.")
      return None


#*******************************************************************************

    def logout_user(self):
      self.current_user = None  # Réinitialiser l'utilisateur actuellement connecté


In [ ]:
import getpass

def main():
    filename = "/content/sample_data/DB/users.csv"

    db_manager = UserDBManager(filename)
    user_manager = UserManager(db_manager)

    # Demande à l'utilisateur s'il souhaite s'inscrire ou se connecter ou se deconnecter
    choice = input("Do you want to register (r) or login (l)?")

    if choice.lower() == 'r':  # Inscription
        username = input("Enter your username: ")
        password = getpass.getpass("Enter your password: ")

        user_manager.register_user(username, password, filename)

    elif choice.lower() == 'l':  # Connexion
        username = input("Enter your username: ")
        password = getpass.getpass("Enter your password: ")

        logged_in = user_manager.login_user(username, password)

        if logged_in:     # Si l'utilisateur est connecté, lui donner la possibilité de se déconnecter

            logout_choice = input("Do you want to logout (y/n)?")
            if logout_choice.lower() == 'y':
                user_manager.logout_user()

    else:
        print("Invalid choice.")

#****************************************

if __name__ == "__main__":
    main()

Do you want to register (r) or login (l)?r
Enter your username: Fatima
Enter your password: ··········
User Fatima was successfully registered.


# **PARTIE 2**

OPRFProtocol: Defines methods to perform OPRF computations both on the client and server sides. It involves generating a hashed password, selecting a random scalar, and computing shared values between the client and server using exponentiation modulo a large prime.

DHKeyExchange: Provides a static method to perform the Diffie-Hellman key exchange between two parties, which allows them to agree on a shared secret key over an insecure channel.

Encryption: Contains methods for encrypting and decrypting messages using symmetric-key encryption with AES-GCM mode. It generates random initialization vectors (IVs) and authentication tags for secure encryption.

DigitalSignature: Implements methods for generating and verifying digital signatures using ECDSA (Elliptic Curve Digital Signature Algorithm) with the SHA-256 hash function.

User: Represents a user in the system with attributes such as username, password, private key, and public key. It provides methods for user registration and login. During registration, the user's password is processed using OPRF, and the resulting shared secret key is used for encryption of the user's private key, which is then signed with the user's private key. During login, the user's credentials are verified, and the stored signature is validated to ensure data integrity.

Main Function: Demonstrates the usage of the implemented functionality by registering a user, generating server keys, and performing user login.


In [ ]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import ec, dh
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os
import getpass

class OPRFProtocol:
    def compute_oprf_client(self, password, salt):
        # Compute H(P)
        hashed_password = hashes.Hash(hashes.SHA256(), backend=default_backend())
        hashed_password.update(password.encode())
        hashed_password_value = hashed_password.finalize()

        # Select a random scalar r
        r = os.urandom(32)

        # Compute C = H(P)^r
        C = pow(int.from_bytes(hashed_password_value, 'big'), int.from_bytes(r, 'big'), 2**2048)

        return hashed_password_value, r, C

    def compute_oprf_server(self, salt, C):
        # Compute R = C^s
        R = pow(C, int.from_bytes(salt, 'big'), 2**2048)
        return R

#*******************************************************************************

class DHKeyExchange:
    @staticmethod
    def perform_dh_key_exchange(private_key, public_key):
        shared_key = private_key.exchange(ec.ECDH(), public_key)
        return shared_key

#*******************************************************************************

class Encryption:
    @staticmethod
    def encrypt_message(key, message):
        iv = os.urandom(16)  # Generate a random IV
        cipher = Cipher(algorithms.AES(key), modes.GCM(iv), backend=default_backend())
        encryptor = cipher.encryptor()
        ciphertext = encryptor.update(message) + encryptor.finalize()
        return ciphertext, encryptor.tag, iv

    @staticmethod
    def decrypt_message(key, ciphertext, tag, iv):
        cipher = Cipher(algorithms.AES(key), modes.GCM(iv, tag), backend=default_backend())
        decryptor = cipher.decryptor()
        plaintext = decryptor.update(ciphertext) + decryptor.finalize()
        return plaintext

#*******************************************************************************

class DigitalSignature:
    @staticmethod
    def sign(private_key, message):
        return private_key.sign(message, ec.ECDSA(hashes.SHA256()))

    @staticmethod
    def verify(public_key, signature, message):
        try:
            public_key.verify(signature, message, ec.ECDSA(hashes.SHA256()))
            return True
        except:
            return False

#*******************************************************************************

class User:
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
        self.public_key = self.private_key.public_key()


    def is_password_strong(self, password):
        if len(password) < 14:
            return False

        has_digit = any(char.isdigit() for char in password)
        has_upper = any(char.isupper() for char in password)
        has_lower = any(char.islower() for char in password)
        has_special = any(char in "!@#$%^&*()-_=+[]{}|;:'\",.<>/?`~" for char in password)

        return has_digit and has_upper and has_lower and has_special


    def register(self, salt, server_public_key):
        while not self.is_password_strong(self.password):
            raise ValueError("Le mot de passe n'est pas suffisamment fort.")

        oprf_client = OPRFProtocol()

        hashed_password, r, C = oprf_client.compute_oprf_client(self.password, salt)

        R = oprf_client.compute_oprf_server(salt, C)

        shared_secret_key = DHKeyExchange.perform_dh_key_exchange(self.private_key, server_public_key)

        self.shared_secret_key = shared_secret_key
        private_key_bytes = self.private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        )

        ciphertext, tag, iv = Encryption.encrypt_message(shared_secret_key, private_key_bytes)

        signature = DigitalSignature.sign(self.private_key, ciphertext)

        return R, C, ciphertext, tag, iv, signature

    def login(self, R, M, server_private_key, server_public_key):
        shared_secret_key = DHKeyExchange.perform_dh_key_exchange(self.private_key, server_public_key)

        plaintext = Encryption.decrypt_message(shared_secret_key, M[0], M[1], M[2])

        if not DigitalSignature.verify(server_public_key, M[3], plaintext):
            raise Exception("Signature verification failed.")
        return plaintext


#*******************************************************************************

def main():
    username = input("Enter your username: ")
    password = getpass.getpass("Enter your password: ")
    salt = os.urandom(32)

    server_private_key = ec.generate_private_key(ec.SECP256R1(), os.urandom(32))
    server_public_key = server_private_key.public_key()

    alice = User(username, password)

    # Enregistrement de l'utilisateur
    R, C, ciphertext, tag, iv, signature = alice.register(salt, server_public_key)

    # Connexion de l'utilisateur
    alice_shared_key = alice.login(R, (ciphertext, tag, iv, signature), server_private_key, server_public_key)
    print(username, "'s shared secret key:", alice_shared_key)

if __name__ == "__main__":
    main()
